# RQ3 Link Analysis: 2020 US Elections 

In [1]:
import json
import pandas as pd
import numpy as np
import networkx as nx

Import donwloaded tweets

In [2]:
with open("data/final_project.json", "rb") as f:
    data = f.readlines()
    data = [json.loads(str_) for str_ in data]

In [3]:
df_tweets = pd.DataFrame.from_records(data)

In [4]:
print("The total number of tweets is: {}" .format(len(df_tweets)))

The total number of tweets is: 311927


a. Generate the retweet graph

In [5]:
# Take the tweets that are a retweet
df_retweets = df_tweets[df_tweets["text"].apply(lambda x: x[:2]) == "RT"]

In [6]:
#Remove tweets that have a corrupted retweeted status
removed_tweets = 0

for pos, tweet in df_retweets.iterrows():
    tweet_id = df_retweets["id"][pos]

    # Compute number of likes and number of retweets for the tweet
    retweeted_status = df_retweets["retweeted_status"][pos]
    if isinstance(retweeted_status,dict) != True:
        df_retweets.drop([pos], inplace=True)
        removed_tweets += 1
                
print("The number of removed tweets is: {}" .format(removed_tweets))

C:\Users\julia\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


The number of removed tweets is: 37


In [7]:
df_graph = pd.DataFrame(columns=["source", "destination"])

# add source-nodes
df_graph["source"] = df_retweets["user"].apply(lambda x: x["screen_name"]) #Retrieve name of user retweeting

# add destination-nodes
df_graph["destination"] = df_retweets["retweeted_status"].apply(lambda x: x["user"]["screen_name"]) #Retrieve name of user that is being retweeted

In [8]:
df_graph.drop_duplicates(inplace=True)
df_graph.head()

,source,destination
0,wabisabine,Amy_Siskind
1,nikriv2,AdamParkhomenko
2,FoxfireStronomy,tara_atrandom
3,haschke_sam,realDonaldTrump
4,LisPG168,realTrumpForce


In [9]:
df_graph.describe()

,source,destination
count,201036,201036
unique,95589,19117
top,Reneami50020561,realDonaldTrump
freq,75,6349


b. Split the edges in train/test, where the test is given by the 20% of the edges of the graph. These edges are sampling by random from the whole edgelist.

In [12]:
# Generate test with 20% of edges
df_retweets_test = df_graph.sample(frac=0.2, random_state=1)

#Generate train the remaining 80% of the edges
df_retweets_train = df_graph.drop(df_retweets_test.index)

In [13]:
df_retweets_test.describe()

,source,destination
count,40207,40207
unique,30229,7223
top,HowyofWI,realDonaldTrump
freq,16,1312


In [14]:
df_retweets_train.describe()

,source,destination
count,160829,160829
unique,82730,16773
top,Tanis42,realDonaldTrump
freq,67,5037


c. Network-based predictions: Train 4 different algorithms to predict the edges in the test. The prediction is supposed to be done only using the edges information and run over only the edges at distance 2 (friends of friends) of the nodes of the test set. So the list of potential recommendations are given only by all the friends-of-friends of the source and destination nodes included in the test-set. The training phase is instead applied over all 80% remaining edges (training-set).

c.1. Adamic-Adar

c.2. Alternative Least Squares

c.3. Pagerank

c.4. Node2vec

d. Generate a top-10 list of recommendations for all the source nodes present in the test-set.

RQ 3A - Which is the best algorithm among the 4 selected in terms of accuracy? HINT - Use the nDCG, plus one other measurest (of your choice) to compare the results.

RQ 3B - Propose a new strategy to predict the links in the test-set. Which is the accuracy of the new algorithm if compared with the previous ones? Explain in detail the strategy you used and prove its effectiveness.